In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [3]:
import pandas as pd
filepath_csv = '/media/sf_VBox_Shared/CaseLaw/graphs/test3 wg aanspr.csv'
links_wgaans = pd.read_csv(filepath_csv, sep=';', header=0)
links_wgaans.columns = ['id', 'reference']
links_wgaans = links_wgaans.drop_duplicates()
ids_wgaans = list(pd.concat([links_wgaans['id'], links_wgaans['reference']]).unique())

In [27]:
node_list = []
for ecli in ids_wgaans:
    querystring = '''
            prefix dcterm: <http://purl.org/dc/terms/>
            select ?type ?id ?to ?title ?creator ?date ?subject ?abstract
            where {
                {
                BIND("node" AS ?type).
                ?id dcterm:type	<http://psi.rechtspraak.nl/uitspraak>.
                filter (?id = <http://deeplink.rechtspraak.nl/uitspraak?id=''' + ecli+'''>).
                optional { ?id dcterm:creator ?creator}.
                   optional { ?id dcterm:abstract ?abstract}.
                   optional { ?id dcterm:subject ?subject}.
                   optional { ?id dcterm:date ?date}.
                   optional { ?id dcterm:title ?title}
                }
                
                union
                {
                BIND("link" AS ?type).
                ?id dcterm:references ?to.
                filter (?id = <http://deeplink.rechtspraak.nl/uitspraak?id=''' + ecli+'''>).
                }
              }
            '''
    #print(querystring)
    sparql.setQuery(querystring)
    sparql.setReturnFormat(JSON)
    ret = sparql.query()
    node_list.append(ret.convert())

In [30]:
nodes = [res for node in node_list for res in node['results']['bindings'] if res['type']['value']=='node']
links = [res for node in node_list for res in node['results']['bindings'] if res['type']['value']=='link']

In [31]:
def nodes_to_json(nodes_in, variables):
    nodes_json = []
    unique_ids = []
    for d in nodes_in:
        if d['id']['value'] not in unique_ids:
            dout = {}
            for var in variables:
                dout[var] = d.get(var,{'value':''})['value']
            nodes_json.append(dout)
            unique_ids.append(d['id']['value'] )
    return nodes_json, unique_ids

In [32]:
variables = [x for x in node_list[0]['head']['vars'] if x not in ['type', 'from', 'to']]
nodes_json, unique_ids = nodes_to_json(nodes, variables)

In [33]:
print(len(links), len(links_wgaans))

177 177


In [34]:
def links_to_json(links_in, node_ids):
    links_json = []
    for d in links_in:
        target = d['to']['value']
        if target in node_ids:
            dout = {'source': d['id']['value'],
                    'target': target }
            dout['id'] = dout
            links_json.append(dout)
    return links_json

In [35]:
references = links_to_json(links, unique_ids)

In [36]:
import json

filename = '/media/sf_VBox_Shared/CaseLaw/graphs/wgaans_sigma_v2.json.json'
#Set id for sigma
for r in references:
    r['id'] = r['source'] + '_' + r['target']

with open(filename, 'w') as outfile:
    json.dump({'nodes': nodes_json, 'edges': references}, fp=outfile)